# Qiskit Assignment 1
## Practice with Pauli Gates and other Qiskit tools
This assignment will...

### Learning Objectives
1. Build Pauli gates from Qiskit's u gate
2. Solve P = NP
3. etc...

In [1]:
#Import Qiskit and other needed packages
from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
import numpy as np
import pprint

#### Task 1 - Constructing Pauli Z gate from u Gate
We can use Qiskit's [u gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.UGate.html) to construct arbitrary quantum operations. Fill in the parameters below so that the u gate implements a Pauli Z gate.

In [2]:
qc_pauli_z = QuantumCircuit(1,1)
theta = 0 # BEGIN SOLUTION
phi = np.pi #BEGIN SOLUTION
lam = 0 # BEGIN SOLUTION
# END SOLUTION
qc_pauli_z.u(theta,phi,lam,0)

In [ ]:
#check statevector and check contains u gate???

#### Task 3 - Using Qiskit's Bloch Sphere
Provide demo cell. Then ask for Bloch sphere image of circuit that implements X gate then Z gate

In [ ]:
#demo bloch sphere code cell

In [ ]:
# partially implemented circuit to add bloch sphere code and 2 gates

Question: Using just the standard Pauli matrices, how many unique points can I travel to on the Bloch sphere?

2

### Task 4 - Arbitrary Rotations
Now let's use the rx, ry, rz gates and see where we can go on the Bloch sphere. Create a circuit and display the Bloch sphere of a circuit that ends up in a certain area on the Bloch sphere.

In [ ]:
# bloch sphere implementation

#### Task 6 - Initializing a quantum state
We can use... the QuantumCircuit's [initialize method](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html#qiskit.circuit.QuantumCircuit.initialize) to prepare a specific initial state. See the demo cell below. Use the initialize method to prepare a qubit in state x.

In [ ]:
# demo cell
qc = QuantumCircuit(1,1)
qc.initialize([0,1],0)
qc.measure(0,0)
qc.draw(output='mpl')

In [ ]:
#solution cell

#### Task 7 - Global Phase
Recognizing phase is important blah blah... Look at the two circuits below. Are these circuits identical? Up to a global phase?

In [ ]:
# demo cell 1

In [ ]:
# demo cell 2

## Conclusion
Any finals thoughts/summary go here...

### Extension Ideas
1. Solve the twin prime conjecture
2. Count down from infinity
3. etc...

In [ ]:
# Task 5 - play with hadamard gate
# Task 6 - easy pauli gate puzzles